Forked from https://www.kaggle.com/suresk/fastai2-starter?scriptVersionId=42023177

In [ ]:
# Install the fastai v2 dataset

!pip uninstall fastai -y
!pip install /kaggle/input/fast-v2-offline/dataclasses-0.6-py3-none-any.whl
!pip install /kaggle/input/fast-v2-offline/torch-1.6.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install /kaggle/input/fast-v2-offline/torchvision-0.7.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install /kaggle/input/fast-v2-offline/fastcore-1.0.1-py3-none-any.whl
!pip install /kaggle/input/fast-v2-offline/fastai-2.0.8-py3-none-any.whl

from fastai.tabular.all import *

import numpy as np
import pandas as pd

In [ ]:
train_x = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
test_x = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_y = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

In [ ]:
cat_names = ['cp_type', 'cp_time', 'cp_dose']
cont_names = [c for c in train_x.columns if c not in cat_names and c != 'sig_id']
y_names = [c for c in train_y.columns if c != 'sig_id']

In [ ]:
train = pd.concat([train_x, train_y], axis=1)

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train))
procs = [Categorify, Normalize]
tab_pan = TabularPandas(train, procs=procs, cat_names=cat_names, cont_names=cont_names, y_names=y_names, splits=splits)

In [ ]:
dls = tab_pan.dataloaders(bs=4096)

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
# learn = tabular_learner(dls, y_range=(0,1), layers=[1024, 512, 256], loss_func=log_loss)

In [ ]:
# metrics = [logloss]

In [ ]:
learn = tabular_learner(dls, y_range=(0,1), layers=[1024, 512, 512, 256], loss_func=BCELossFlat())#, metrics = logloss)

In [ ]:
learn.lr_find()

In [ ]:
lr = 9e-3
learn.fit_one_cycle(10, slice(lr/(2.6**4),lr))#, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(5, slice(lr/(2.6**4),lr))
learn.unfreeze()

In [ ]:
lr = 4e-2
learn.fit_one_cycle(10, slice(lr/(2.6**4),lr))

In [ ]:

learn.fit_one_cycle(20, slice(1e-4, 8e-3))

In [ ]:
learn.fit_one_cycle(25, slice(8e-4, 8e-3))

In [ ]:
learn.fit_one_cycle(30, slice(5e-4, 5e-3))

In [ ]:
test_dl = learn.dls.test_dl(test_x)
sub = learn.get_preds(dl=test_dl)

In [ ]:
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
submission[[c for c in submission.columns if c != 'sig_id']] = sub[0]

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)